In [41]:
# Importar librerías necesarias
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import datetime
import json

In [42]:
video_dir = 'C:\\Tesis\\TestErgo\\muestra'

In [43]:
def load_videos(video_dir):
    videos = []
    for filename in os.listdir(video_dir):
        if filename.endswith(".mp4"):
            videos.append(os.path.join(video_dir, filename))
    return videos

In [44]:
# Cargar videos
videos = load_videos(video_dir)

In [45]:
# Función para extraer frames de los videos
def extract_frames(video_path, interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames

In [46]:
# Obtener y mostrar los nombres de todos los videos
for video_path in videos:
    video_name = os.path.basename(video_path)
    print(video_name)

Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p).mp4


In [47]:
# Inicializar MediaPipe para detección de puntos clave
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Función para calcular el ángulo entre tres puntos
def calculate_angle(a, b, c, d=None):
    if d is None:
        # Calcula el ángulo entre a, b y c
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto

        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle
    else:
        # Calcula el ángulo entre a, b, c y d
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto
        d = np.array(d)  # Cuarto punto

        radians = np.arctan2(d[1] - c[1], d[0] - c[0]) - np.arctan2(b[1] - a[1], b[0] - a[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle


In [48]:
# Función para detectar puntos clave usando MediaPipe
def detect_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
            keypoints_list.append(keypoints)
        else:
            keypoints_list.append([])  # Agregar una lista vacía si no se detectan keypoints

    cap.release()
    pose.close()
    return keypoints_list

In [49]:
# Función para analizar videos masivos y guardar resultados en un solo JSON
'''def analyze_videos_in_folder(video_folder, output_json_path):
    all_analysis_results = []
    
    for video_file in os.listdir(video_folder):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(video_folder, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            keypoints_list = detect_keypoints(video_path)
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)
            all_analysis_results.extend(analysis_results)
            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)
    
    combined_json_path = os.path.join(json_folder, 'Resultado_analisis_combinados.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(all_analysis_results, json_file, indent=4)

    print(f"Resultados del análisis exportados a: {output_json_path}")

# Ejemplo de uso:
#video_folder = 'C:\\Tesis\\TestErgo\\muestra'
video_folder = video_dir #'C:\\Tesis\\TestErgo\\VIDEOS MAQUILAS'
output_json_path = 'C:\\Tesis\\TestErgo\\analisis_videos.json'

analyze_videos_in_folder(video_folder, output_json_path)'''


'def analyze_videos_in_folder(video_folder, output_json_path):\n    all_analysis_results = []\n    \n    for video_file in os.listdir(video_folder):\n        if video_file.endswith((\'.mp4\', \'.avi\', \'.mov\')):\n            video_path = os.path.join(video_folder, video_file)\n            video_name = os.path.splitext(video_file)[0]\n            \n            keypoints_list = detect_keypoints(video_path)\n            cap = cv2.VideoCapture(video_path)\n            fps = int(cap.get(cv2.CAP_PROP_FPS))\n            cap.release()\n\n            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)\n            all_analysis_results.extend(analysis_results)\n            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)\n    \n    combined_json_path = os.path.join(json_folder, \'Resultado_analisis_combinados.json\')\n    with open(output_json_path, \'w\') as json_file:\n        json.dump(all_analysis_results, json_file,

In [50]:
def analyze_keypoints(video_path):
    keypoints_list = detect_keypoints(video_path)
    fps = None
    analysis_results = []

    for frame_idx, keypoints in enumerate(keypoints_list):
        second = frame_idx // fps if fps else 0

        if not keypoints:
            analysis_results.append({
                'segundo': second,
                'frame': frame_idx,
                'analisis_video': video_path,
                'angulo_hombro_izquierdo': 0,
                'angulo_del_hombro_derecho': 0,
                'angulo_codo_izquierdo': 0,
                'angulo_codo_derecho': 0,
                'angulo_de_muneca_izquierda': 0,
                'angulo_de_muneca_derecha': 0,
                'angulo_mano_izquierdo': 0,
                'angulo_mano_derecho': 0,
                'posicion_hombro_izquierdo': 0,
                'posicion_hombro_derecho': 0,
                'posicion_codo_izquierdo': 0,
                'posicion_codo_derecho': 0,
                'posicion_muneca_izquierda': 0,
                'posicion_muneca_derecha': 0,
                'posicion_mano_izquierda': 0,
                'posicion_mano_derecha': 0,
                'keypoints': []
            })
            continue

        # Extraer puntos clave izquierdos
        left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_elbow = keypoints[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
        left_hand = keypoints[mp_pose.PoseLandmark.LEFT_INDEX.value]

        # Extraer puntos clave derechos
        right_shoulder = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        right_elbow = keypoints[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        right_wrist = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        right_hand = keypoints[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        # Calcular ángulos izquierdos
        left_shoulder_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        left_elbow_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_hand_angle = calculate_angle(left_elbow, left_wrist, left_hand)

        # Calcular ángulos derechos
        right_shoulder_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_elbow_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_hand_angle = calculate_angle(right_elbow, right_wrist, right_hand)

        # Si no se ha establecido aún el FPS, se establece ahora
        if fps is None:
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

        analysis_results.append({
            'segundo': second,
            'frame': frame_idx,
            'analisis_video': video_path,
            'angulo_hombro_izquierdo': left_shoulder_angle,
            'angulo_del_hombro_derecho': right_shoulder_angle,
            'angulo_codo_izquierdo': left_elbow_angle,
            'angulo_codo_derecho': right_elbow_angle,
            'angulo_de_muneca_izquierda': left_wrist_angle,
            'angulo_de_muneca_derecha': right_wrist_angle,
            'angulo_mano_izquierdo': left_hand_angle,
            'angulo_mano_derecho': right_hand_angle,
            'posicion_hombro_izquierdo': left_shoulder,
            'posicion_hombro_derecho': right_shoulder,
            'posicion_codo_izquierdo': left_elbow,
            'posicion_codo_derecho': right_elbow,
            'posicion_muneca_izquierda': left_wrist,
            'posicion_muneca_derecha': right_wrist,
            'posicion_mano_izquierda': left_hand,
            'posicion_mano_derecha': right_hand,
            'keypoints': keypoints
        })

    return analysis_results, fps

In [51]:
def draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder):
    analysis_results, fps = analyze_keypoints(video_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frame_folder = os.path.join(output_folder, video_name)
    output_video_name = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    output_video_path = os.path.join(output_video_folder, output_video_name)
    json_filename = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    json_path = os.path.join(json_folder, json_filename)

    # Crear carpetas si no existen
    if not os.path.exists(video_frame_folder):
        os.makedirs(video_frame_folder)
    if not os.path.exists(output_video_folder):
        os.makedirs(output_video_folder)
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)

    # Crear el video marcado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    for i in range(len(analysis_results)):
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = analysis_results[i]['keypoints']
        left_shoulder_angle = analysis_results[i]['angulo_hombro_izquierdo']
        right_shoulder_angle = analysis_results[i]['angulo_del_hombro_derecho']
        left_elbow_angle = analysis_results[i]['angulo_codo_izquierdo']
        right_elbow_angle = analysis_results[i]['angulo_codo_derecho']
        left_wrist_angle = analysis_results[i]['angulo_de_muneca_izquierda']
        right_wrist_angle = analysis_results[i]['angulo_de_muneca_derecha']
        left_hand_angle = analysis_results[i]['angulo_mano_izquierdo']
        right_hand_angle = analysis_results[i]['angulo_mano_derecho']

        # Dibujar puntos clave y conexiones con el color original
        if keypoints:
            for j, point in enumerate(keypoints):
                cv2.circle(frame, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, (0, 255, 0), -1)

                # Añadir texto con información relevante junto a cada punto
                if j == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro izquierdo: {int(left_shoulder_angle) if left_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro derecho: {int(right_shoulder_angle) if right_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo izquierdo: {int(left_elbow_angle) if left_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo derecho: {int(right_elbow_angle) if right_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca izquierda: {int(left_wrist_angle) if left_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca derecha: {int(right_wrist_angle) if right_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano izquierda: {int(left_hand_angle) if left_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano derecha: {int(right_hand_angle) if right_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

        # Guardar el frame en la carpeta de salida con el nombre del video seguido de su numeración
        output_frame_path = os.path.join(video_frame_folder, f'{video_name}_frame_{frame_count:04d}.png')
        cv2.imwrite(output_frame_path, frame)
        frame_count += 1

        # Guardar el frame en el video marcado
        out.write(frame)

    cap.release()
    out.release()
    
    # Guardar el archivo JSON con los resultados del análisis
    with open(json_path, 'w') as json_file:
        json.dump(analysis_results, json_file, indent=4)

    print(f"Frames exportados a: {video_frame_folder}")
    print(f"Video exportado a: {output_video_path}")
    print(f"JSON exportado a: {json_path}")
    return video_frame_folder, output_video_path, json_path


In [52]:
video_path = videos[0]
output_folder = 'C:\\Tesis\\TestErgo\\resultados'
output_video_folder = 'C:\\Tesis\\TestErgo\\videoMarcado'
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'

draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder)

Frames exportados a: C:\Tesis\TestErgo\resultados\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)
Video exportado a: C:\Tesis\TestErgo\videoMarcado\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.mp4
JSON exportado a: C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json


('C:\\Tesis\\TestErgo\\resultados\\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)',
 'C:\\Tesis\\TestErgo\\videoMarcado\\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.mp4',
 'C:\\Tesis\\TestErgo\\archivos_json\\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json')

In [53]:
import os
import glob
import json
import cv2

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

def get_duration_from_video(video_path):
    """Calcula la duración total del video en minutos."""
    try:
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            raise ValueError(f"No se pudo abrir el video en {video_path}.")
        fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        duration_sec = total_frames / fps
        video.release()
        return duration_sec / 60  # Convertir a minutos
    except Exception as e:
        raise RuntimeError(f"Error al calcular la duración del video: {e}")

def get_non_repetitive_work_time(video_data):
    """Obtiene el tiempo de trabajo no repetitivo en minutos."""
    try:
        non_repetitive_time = sum(activity['duration'] for activity in video_data['activities'] if not activity['is_repetitive'])
        return non_repetitive_time / 60  # Convertir a minutos
    except KeyError as e:
        raise KeyError(f"Falta una clave en los datos del video: {e}")

def get_breaks_and_rest_time(video_data):
    """Identifica el tiempo de pausas y descansos en minutos."""
    try:
        pauses = sum(break_time['duration'] for break_time in video_data.get('breaks', []))
        rest = sum(rest_time['duration'] for rest_time in video_data.get('rests', []))
        return pauses / 60, rest / 60  # Convertir a minutos
    except KeyError as e:
        raise KeyError(f"Falta una clave en los datos del video: {e}")

def get_number_of_cycles(video_data):
    """Cuenta el número de ciclos de trabajo."""
    try:
        return video_data.get('cycles_count', 0)
    except KeyError as e:
        raise KeyError(f"Falta una clave en los datos del video: {e}")

# Encuentra el archivo JSON y el video más recientes
json_folder = 'C:/Tesis/TestErgo/archivos_json'
video_folder = 'C:/Tesis/TestErgo/videoMarcado'

json_path = get_most_recent_file(json_folder, '.json')
video_path = get_most_recent_file(video_folder, '.mp4')

if json_path is None or video_path is None:
    raise FileNotFoundError("No se encontraron archivos JSON o de video en las carpetas especificadas.")

print(f"Archivo JSON más reciente: {json_path}")
print(f"Video más reciente: {video_path}")

# Extraer datos del video
duration_total = get_duration_from_video(video_path)
video_data = {
    'activities': [{'duration': 480, 'is_repetitive': True}],  # Ejemplo de datos, reemplazar con tu análisis real
    'breaks': [{'duration': 60}],  # Ejemplo de datos
    'rests': [{'duration': 60}],  # Ejemplo de datos
    'cycles_count': 9000  # este campo quedará fijo con 9000 ciclos
}

# Calcular TNTR y TNC
try:
    DT = duration_total
    TNR = get_non_repetitive_work_time(video_data)
    P, A = get_breaks_and_rest_time(video_data)
    NC = get_number_of_cycles(video_data)

    if NC == 0:
        raise ValueError("El número de ciclos no puede ser cero.")

    TNTR = DT - (TNR + P + A)
    TNC = 60 * TNTR / NC

    # Leer el archivo JSON existente
    with open(json_path, 'r') as file:
        data = json.load(file)

    # Verificar si el contenido es una lista
    '''if isinstance(data, list):
        for item in data:
            item['Tiempo Neto de Trabajo Repetitivo'] = TNTR
            item['Tiempo Neto del Ciclo'] = TNC
    else:
        data['Tiempo Neto de Trabajo Repetitivo'] = TNTR
        data['Tiempo Neto del Ciclo'] = TNC

    # Guardar el JSON actualizado
    with open(json_path, 'w') as file:
        json.dump(data, file, indent=4)

    print("El JSON se ha actualizado con TNTR y TNC.")
except Exception as e:
    print(f"Error al procesar el archivo JSON o al calcular TNTR/TNC: {e}")'''
# Verificar si el contenido es una lista o un diccionario
    if isinstance(data, list):
        for item in data:
            item['Tiempo Neto de Trabajo Repetitivo'] = TNTR
            item['Tiempo Neto del Ciclo'] = TNC
    elif isinstance(data, dict):
        data['Tiempo Neto de Trabajo Repetitivo'] = TNTR
        data['Tiempo Neto del Ciclo'] = TNC
    else:
        raise ValueError("El contenido del archivo JSON no es ni una lista ni un diccionario")

    # Guardar el JSON actualizado
    with open(json_path, 'w') as file:
        json.dump(data, file, indent=4)

    print("El JSON se ha actualizado con TNTR y TNC.")
except Exception as e:
    print(f"Error al procesar el archivo JSON o al calcular TNTR/TNC: {e}")

Archivo JSON más reciente: C:/Tesis/TestErgo/archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json
Video más reciente: C:/Tesis/TestErgo/videoMarcado\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.mp4
El JSON se ha actualizado con TNTR y TNC.


Factor de Recuperación

In [54]:
import json
import re

def calculate_fr(pauses, lunch_break_duration, shift_duration):
    """
    Cálculo de puntuación del FR basado en la Tabla 1.
    """
    num_pauses = len(pauses)
    is_lunch_break_present = lunch_break_duration > 0

    if is_lunch_break_present and lunch_break_duration >= 8:
        if shift_duration >= 60:
            return 0  # Ideal
        elif num_pauses >= 4:
            return 2
    elif is_lunch_break_present and num_pauses >= 3:
        return 3
    elif num_pauses >= 2:
        return 4
    elif num_pauses == 1:
        return 6
    else:
        return 10  # Sin pausas adecuadas

def convert_to_minutes(time_str):
    """Convierte una cadena de tiempo en formato 'Xh Ym' a minutos y valida el formato."""
    time_str = time_str.strip()
    if re.match(r'^\d+h \d+m$', time_str):
        hours, minutes = time_str.split('h')
        minutes = minutes.replace('m', '').strip()
        return int(hours.strip()) * 60 + int(minutes)
    elif re.match(r'^\d+h$', time_str):
        hours = time_str.replace('h', '').strip()
        return int(hours) * 60
    elif re.match(r'^\d+m$', time_str):
        minutes = time_str.replace('m', '').strip()
        return int(minutes)
    else:
        raise ValueError("Formato de tiempo no válido. Use 'Xh Ym', 'Xh' o 'Ym'.")

def get_input(prompt):
    """Solicita al usuario un valor y maneja errores de entrada."""
    while True:
        try:
            value = input(prompt)
            return convert_to_minutes(value)
        except ValueError as e:
            print(f"Error: {e}. Inténtalo de nuevo con el formato correcto.")

# Solicitar datos al usuario
print("Formato de tiempo: 'Xh Ym', 'Xh' o 'Ym' (ejemplo: '8h 30m', '1h', '45m')")

""" shift_duration = get_input("Ingrese la duración del turno (en horas y/o minutos): ")
lunch_break_duration = get_input("Ingrese la duración del tiempo de almuerzo (en horas y/o minutos): ")

num_pauses = int(input("Ingrese el número total de pausas: "))
pauses = [] """
# Datos de prueba
pauses = []  # Pausas en minutos
lunch_break_duration = 60  # Tiempo de almuerzo en minutos
shift_duration = 480  # Duración del turno en minutos
num_pauses = 3

""" for i in range(num_pauses):
    pause_duration_str = get_input(f"Ingrese la duración de la pausa {i+1} (en horas y/o minutos): ")
    pauses.append(pause_duration_str) """

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'

'''def get_most_recent_file(folder_path, extension):
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)'''

# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]
# Calcular el FR
# Extraer datos del JSON
# Los datos se sobreescriben con los valores ingresados por el usuario
# Calcular el FR
fr = calculate_fr(pauses, lunch_break_duration, shift_duration)

# Añadir el cálculo al JSON
data['Factor de Recuperacion'] = fr

# Guardar el JSON actualizado
'''updated_json_path = json_path.replace('.json', 'FR.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)'''
# Guardar el JSON actualizado (sobrescribiendo el archivo existente)
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)
print(f"El JSON se ha actualizado con el Factor de Recuperación y guardado en {json_path}.")
#print(f"El JSON se ha actualizado con el Factor de Recuperación y guardado en {updated_json_path}.")

Formato de tiempo: 'Xh Ym', 'Xh' o 'Ym' (ejemplo: '8h 30m', '1h', '45m')
El JSON se ha actualizado con el Factor de Recuperación y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


Factor de frecuencia

In [55]:
import json

def calcular_ff(atd, ate):
    """
    Calcula el Factor de Frecuencia (FF) basado en las puntuaciones ATD y ATE.
    
    :param atd: Puntuación de Acciones Técnicas Dinámicas
    :param ate: Puntuación de Acciones Técnicas Estáticas
    :return: Puntuación del Factor de Frecuencia (FF)
    """
    return max(atd, ate)

def obtener_valor_ingresado(mensaje, valor_por_defecto=0):
    """
    Solicita al usuario ingresar un valor y valida la entrada.
    
    :param mensaje: Mensaje a mostrar al usuario
    :param valor_por_defecto: Valor a utilizar si no se ingresa ningún valor
    :return: Valor ingresado por el usuario
    """
    while True:
        try:
            entrada = input(mensaje)
            if entrada.strip() == '':
                return valor_por_defecto
            valor = float(entrada)
            if valor < 0:
                print("Por favor, ingrese un valor positivo.")
            else:
                return valor
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número válido.")

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

""" # Intentar extraer puntuaciones del JSON
atd = data.get('AccionesTecnicasDinámicas', None)
ate = data.get('AccionesTecnicasEstáticas', None) """

# Definir valores predeterminados para ATD y ATE
valor_atd = 0.0  # Valor fijo de ejemplo para Acciones Técnicas Dinámicas   
valor_ate = 0.0  # Valor fijo de ejemplo para Acciones Técnicas Estáticas

# Intentar extraer puntuaciones del JSON, sino usar valores predeterminados
atd = data.get('AccionesTecnicasDinámicas', valor_atd)
ate = data.get('AccionesTecnicasEstáticas', valor_ate)
               
""" # Si no están presentes en el JSON, pedir al usuario que ingrese los valores
if atd is None or ate is None:
    print("Algunos valores no se encontraron en el archivo JSON.")
    if atd is None:
        atd = obtener_valor_ingresado("Ingrese la puntuación de Acciones Técnicas Dinámicas (ATD): ")
    if ate is None:
        ate = obtener_valor_ingresado("Ingrese la puntuación de Acciones Técnicas Estáticas (ATE): ")
 """
# Calcular el FF
ff = calcular_ff(atd, ate)

# Añadir el cálculo al JSON
data['FF'] = ff

# Guardar el JSON actualizado
'''updated_json_path = json_path.replace('.json', '__ff.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Frecuencia y guardado en {updated_json_path}.")'''
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Frecuencia y guardado en {json_path}.")

El JSON se ha actualizado con el Factor de Frecuencia y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


In [56]:
import json

def calcular_ffz(acciones):
    """
    Calcula el Factor de Fuerza (FFz) basado en las acciones que requieren fuerza.
    
    :param acciones: Lista de acciones que requieren fuerza con sus detalles
    :return: Puntuación del Factor de Fuerza (FFz)
    """
    puntuacion_total = 0
    
    for accion in acciones:
        esfuerzo = accion['esfuerzo_cr10']
        duracion = accion['duracion']
        
        if esfuerzo in [3, 4]:  # Fuerza Moderada
            if duracion == "1/3 del tiempo":
                puntuacion_total += 2
            elif duracion == "50% del tiempo":
                puntuacion_total += 4
            elif duracion == "> 50% del tiempo":
                puntuacion_total += 6
            elif duracion == "Casi todo el tiempo":
                puntuacion_total += 8
                
        elif esfuerzo in [5, 6]:  # Fuerza Intensa
            if duracion == "2 seg. cada 10 min.":
                puntuacion_total += 4
            elif duracion == "1% del tiempo":
                puntuacion_total += 8
            elif duracion == "5% del tiempo":
                puntuacion_total += 16
            elif duracion == "> 10% del tiempo":
                puntuacion_total += 24
        
        elif esfuerzo >= 7:  # Fuerza Casi Máxima
            if duracion == "2 seg. cada 10 min.":
                puntuacion_total += 6
            elif duracion == "1% del tiempo":
                puntuacion_total += 12
            elif duracion == "5% del tiempo":
                puntuacion_total += 24
            elif duracion == "> 10% del tiempo":
                puntuacion_total += 32
    
    return puntuacion_total

def obtener_acciones_predeterminadas():
    """
    Proporciona una lista de acciones predefinidas con sus detalles.
    
    :return: Lista de acciones con sus detalles
    """
    acciones = [
        {'esfuerzo_cr10': 2, 'duracion': "1/3 del tiempo"},
        #{'esfuerzo_cr10': 4, 'duracion': "50% del tiempo"},
        #{'esfuerzo_cr10': 5, 'duracion': "2 seg. cada 10 min."},
        #{'esfuerzo_cr10': 6, 'duracion': "1% del tiempo"},
        #{'esfuerzo_cr10': 7, 'duracion': "5% del tiempo"},
        #{'esfuerzo_cr10': 8, 'duracion': "> 10% del tiempo"}
    ]
    return acciones

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Intentar extraer acciones del JSON
acciones = data.get('acciones_fuerza', None)

# Si no están presentes en el JSON, usar acciones predeterminadas
if acciones is None:
    print("No se encontraron acciones de fuerza en el archivo JSON. Usando valores predeterminados.")
    acciones = obtener_acciones_predeterminadas()

# Calcular el FFz
ffz = calcular_ffz(acciones)

# Añadir el cálculo al JSON
data['Factor de Fuerza'] = ffz

# Guardar el JSON actualizado
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Fuerza y guardado en {json_path}.")

No se encontraron acciones de fuerza en el archivo JSON. Usando valores predeterminados.
El JSON se ha actualizado con el Factor de Fuerza y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


In [57]:
import json

def calculate_posture_movements(json_data):
    """
    Calcula el Factor de Posturas y Movimientos (FP) basado en los ángulos de postura del JSON proporcionado.
    """
    
    def get_posture_score(angle):
        """ Determina la puntuación basada en el ángulo de postura """
        # Asegúrate de que angle no sea None; usa 0 como valor predeterminado si es None
        if angle is None:
            angle = 0
            
        if angle < 45:
            return 1  # Buen rango
        elif angle < 90:
            return 2  # Rango moderado
        elif angle < 135:
            return 4  # Rango alto
        else:
            return 8  # Rango muy alto

    # Obtener las puntuaciones individuales basadas en los ángulos
    ph = get_posture_score(json_data.get('angulo_hombro_izquierdo', 0))
    pc = get_posture_score(json_data.get('angulo_codo_izquierdo', 0))
    pm = get_posture_score(json_data.get('angulo_de_muneca_izquierda', 0))
    pa = get_posture_score(json_data.get('angulo_mano_izquierdo', 0))

    # Obtener la puntuación para movimientos estereotipados (asumimos que es un campo existente)
    pes = json_data.get('MovimientosEstereotipados', 0)

    # Calcular FP
    fp = max(ph, pc, pm, pa) + pes

    return fp
def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Calcular el FP
fp = calculate_posture_movements(data)

# Añadir el cálculo al JSON
data['Factor de Posturas y Movimientos'] = fp

# Guardar el JSON actualizado
'''updated_json_path = json_path.replace('.json', '_fp.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Posturas y Movimientos y guardado en {updated_json_path}.")''' 
# Guardar el JSON actualizado (sobrescribiendo el archivo existente)
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Posturas y Movimientos y guardado en {json_path}.")


El JSON se ha actualizado con el Factor de Posturas y Movimientos y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


In [58]:
import json

def calculate_additional_risks(json_data):
    """
    Calcula el Factor de Riesgos Adicionales (FC) basado en los riesgos adicionales del JSON proporcionado.
    """
    # Definir criterios para calcular el FC
    # Ejemplo: Puedes tener varios campos que influyen en el FC
    riesgo_adicional_1 = json_data.get('RiesgoAdicional1', 2)
    riesgo_adicional_2 = json_data.get('RiesgoAdicional2', 1)
    
    # Ejemplo de fórmula para el FC
    fc = riesgo_adicional_1 * 0.2 + riesgo_adicional_2 * 0.2
    
    return fc

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Calcular el FC
fc = calculate_additional_risks(data)

# Añadir el cálculo al JSON
data['Factor de Riesgos Adicionales'] = fc

# Guardar el JSON actualizado
'''updated_json_path = json_path.replace('.json', '_fc.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Riesgos Adicionales (FC) y guardado en {updated_json_path}.")''' 
# Guardar el JSON actualizado (sobrescribiendo el archivo existente)
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Riesgos Adicionales (FC) y guardado en {json_path}.")

El JSON se ha actualizado con el Factor de Riesgos Adicionales (FC) y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


In [59]:
import json

def calcular_md(tntr):
    """
    Calcula el Multiplicador de Duración (MD) basado en el Tiempo Neto de Trabajo Repetitivo (TNTR).
    
    :param tntr: Tiempo Neto de Trabajo Repetitivo en minutos
    :return: Puntuación del Multiplicador de Duración (MD)
    """
    if tntr <= 60:
        return 0.5
    elif 60 < tntr <= 120:
        return 0.5
    elif 120 < tntr <= 180:
        return 0.65
    elif 180 < tntr <= 240:
        return 0.75
    elif 240 < tntr <= 300:
        return 0.85
    elif 300 < tntr <= 360:
        return 0.925
    elif 360 < tntr <= 420:
        return 0.95
    elif 420 < tntr <= 480:
        return 1
    elif 480 < tntr <= 539:
        return 1.2
    elif 539 < tntr <= 599:
        return 1.5
    elif 599 < tntr <= 659:
        return 2
    elif 659 < tntr <= 719:
        return 2.8
    elif tntr >= 720:
        return 4
    else:  # Para análisis multitarea
        if tntr <= 1.87:
            return 0.01
        elif 1.87 < tntr <= 3.75:
            return 0.02
        elif 3.75 < tntr <= 7.5:
            return 0.05
        elif 7.5 < tntr <= 15:
            return 0.1
        elif 15 < tntr <= 30:
            return 0.2
        elif 30 < tntr <= 59:
            return 0.35

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    import os
    import glob
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Extraer TNTR del JSON
tntr = data.get('Tiempo Neto de Trabajo Repetitivo', 480)  # Usar 480 minutos por defecto si no está presente

# Calcular el MD
md = calcular_md(tntr)

# Añadir el cálculo al JSON
data['Multiplicador de Duracion'] = md

# Guardar el JSON actualizado
'''updated_json_path = json_path.replace('.json', '_md.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Multiplicador de Duración y guardado en {updated_json_path}.")''' 
# Guardar el JSON actualizado (sobrescribiendo el archivo existente)
with open(json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Multiplicador de Duración y guardado en {json_path}.")

El JSON se ha actualizado con el Multiplicador de Duración y guardado en C:\Tesis\TestErgo\archivos_json\Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092217.json.


exportar excel solo con resultado checklist OCRA

In [60]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
import os
from datetime import datetime

# Leer el JSON y convertirlo a un DataFrame de pandas
df = pd.DataFrame(data_list)

# Obtener valores para el cálculo de ICKL
fr = df['Factor de Recuperacion'].mean()  # Promedio de los valores si hay múltiples registros
ff = df['FF'].mean()
ffz = df['Factor de Fuerza'].mean()
fp = df['Factor de Posturas y Movimientos'].mean()
fc = df['Factor de Riesgos Adicionales'].mean()
md = df['Multiplicador de Duracion'].mean()

# Calcular el Índice Check List OCRA (ICKL)
ickl = (fr + ff + ffz + fp + fc) * md

# Determinar Nivel de Riesgo y Acción Recomendada
def determinar_nivel_riesgo(ickl):
    if ickl <= 5:
        return 'Óptimo', 'No se requiere', '≤ 1.5'
    elif 5.1 <= ickl <= 7.5:
        return 'Aceptable', 'No se requiere', '1.6 - 2.2'
    elif 7.6 <= ickl <= 11:
        return 'Incierto', 'Se recomienda un nuevo análisis o mejora del puesto', '2.3 - 3.5'
    elif 11.1 <= ickl <= 14:
        return 'Inaceptable Leve', 'Se recomienda mejora del puesto, supervisión médica y entrenamiento', '3.6 - 4.5'
    elif 14.1 <= 22.5:
        return 'Inaceptable Medio', 'Se recomienda mejora del puesto, supervisión médica y entrenamiento', '4.6 - 9'
    else:
        return 'Inaceptable Alto', 'Se recomienda mejora del puesto, supervisión médica y entrenamiento', '> 9'

nivel_riesgo, accion_recomendada, indice_ocra_equivalente = determinar_nivel_riesgo(ickl)

# Obtener el nombre del video y la fecha actual
nombre_video = df['analisis_video'][0].split('\\')[-1].replace('.mp4', '')
fecha_hora = datetime.now().strftime('%Y%m%d_%H%M%S')

# Construir el nombre del archivo Excel
archivo_excel = f'analisisOcra_{nombre_video}_{fecha_hora}.xlsx'

# Definir la ruta de la carpeta 'resultados' dentro de 'static'
static_folder = 'static'
resultados_folder = os.path.join(static_folder, 'resultados')

# Crear la carpeta 'resultados' si no existe
if not os.path.exists(resultados_folder):
    os.makedirs(resultados_folder)

# Definir la ruta del archivo Excel en la carpeta 'resultados'
excel_path = os.path.join(resultados_folder, archivo_excel)

# Crear un nuevo libro de trabajo
wb = Workbook()
ws = wb.active
ws.title = "Resumen OCRA"

# Escribir encabezados para los factores
headers_factores = [
    "Factor de Recuperación (FR)", "Factor de Frecuencia (FF)", 
    "Factor de Fuerza (FFz)", "Factor de Posturas y Movimientos (FP)",
    "Factor de Riesgos Adicionales (FC)", "Multiplicador de Duración (MD)"
]
ws.append(headers_factores)

# Escribir la fila de factores
factores = [fr, ff, ffz, fp, fc, md]
ws.append(factores)

# Escribir encabezados para los resultados del Check List OCRA
ws.append([])  # Añadir una fila en blanco para separación
headers_resultados = [
    "Índice Check List OCRA", "Nivel de Riesgo", "Acción Recomendada", "Índice OCRA Equivalente"
]
ws.append(headers_resultados)

# Escribir la fila de resultados
resultados = [ickl, nivel_riesgo, accion_recomendada, indice_ocra_equivalente]
ws.append(resultados)

# Estilos para la hoja de cálculo
header_fill = PatternFill(start_color='4C47EA', end_color='4C47EA', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
alignment = Alignment(horizontal='center', vertical='center')

# Aplicar estilo a los encabezados de factores
for col_num in range(1, 7):
    cell = ws.cell(row=1, column=col_num)
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = alignment

# Aplicar estilo a los encabezados de resultados
for col_num in range(1, 5):
    cell = ws.cell(row=4, column=col_num)
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = alignment

# Aplicar formato a las celdas de datos
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.alignment = alignment

        # Aplicar formato condicional para los factores en la fila 2
        if cell.row == 2:  # Fila de factores
            if isinstance(cell.value, (int, float)):
                if cell.value > 14:
                    # Color: Rojo claro (FF9999)
                    cell.fill = PatternFill(start_color='FF9999', end_color='FF9999', fill_type='solid')
                elif cell.value > 7.5:
                    # Color: Amarillo claro (FFCC99)
                    cell.fill = PatternFill(start_color='FFCC99', end_color='FFCC99', fill_type='solid')

        # Aplicar formato condicional para los resultados en la fila 5
        if cell.row == 5:  # Fila de resultados
            if isinstance(cell.value, (int, float)):
                if cell.value > 14:
                    # Color: Rojo claro (FF9999)
                    cell.fill = PatternFill(start_color='FF9999', end_color='FF9999', fill_type='solid')
                elif cell.value > 7.5:
                    # Color: Amarillo claro (FFCC99)
                    cell.fill = PatternFill(start_color='FFCC99', end_color='FFCC99', fill_type='solid')


# Ajustar el ancho de las columnas
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass    
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# Guardar el archivo Excel
wb.save(excel_path)

print(f"El Excel se ha creado y guardado en {excel_path}.")

El Excel se ha creado y guardado en static\resultados\analisisOcra_Y2meta.app-Diagnóstico Ergonómico biomecánico_ evaluación final-(480p)_20240805_092224.xlsx.
